# PEPITA network

In [ ]:
# Install torchattacks library if not installed
!pip install torchattacks

#### This code defines and trains the PEPITA neural network for MNIST classification, using adversarial attacks for robustness evaluation. 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import numpy as np
import torch.cuda
import torchattacks

# Define the PEPITA network with flexible hyperparameters
class PEPITANetwork(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate=0.1):
        super(PEPITANetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0], bias=False)
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1], bias=False)
        self.fc3 = nn.Linear(hidden_sizes[1], output_size, bias=False)
        self.dropout_rate = dropout_rate
        
        # He initialization
        fc1_limit = np.sqrt(6.0 / input_size)
        torch.nn.init.uniform_(self.fc1.weight, a=-fc1_limit, b=fc1_limit)
        fc2_limit = np.sqrt(6.0 / hidden_sizes[0])
        torch.nn.init.uniform_(self.fc2.weight, a=-fc2_limit, b=fc2_limit)
        fc3_limit = np.sqrt(6.0 / hidden_sizes[1])
        torch.nn.init.uniform_(self.fc3.weight, a=-fc3_limit, b=fc3_limit)

    def forward(self, x, do_masks=None):
        x = F.relu(self.fc1(x))
        if do_masks is not None:
            x = x * do_masks[0]  # Apply custom dropout mask
        x = F.relu(self.fc2(x))
        if do_masks is not None:
            x = x * do_masks[1]  # Apply custom dropout mask
        x = F.softmax(self.fc3(x), dim=1)
        return x

# Initialize feedback matrix (B)
def initialize_feedback_matrix(input_size, output_size, scale=0.05, device='cpu'):
    sd = np.sqrt(6.0 / input_size)
    B = (torch.rand(input_size, output_size) * 2 * sd - sd) * scale
    return B.to(device)

# Setup data loaders (MNIST dataset)
def get_dataloaders(batch_size=16):
    transform = transforms.Compose([transforms.ToTensor()])
    trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
    testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)
    return trainloader, testloader

# Validation function to evaluate model performance
def validate_pepita(net, dataloader, device, criterion):
    net.eval()  # Set network to evaluation mode
    running_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, target in dataloader:
            inputs = inputs.view(inputs.size(0), -1).to(device)  # Flatten inputs
            target = target.to(device)
            outputs = net(inputs)  # Forward pass
            loss = criterion(outputs, target)
            running_loss += loss.item()

            # Accuracy calculation
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    
    accuracy = 100 * correct / total
    return running_loss / len(dataloader), accuracy

# Adversarial attack evaluation function
def evaluate_model(net, dataloader, device, attack_method=None):
    """
    Evaluate the model on clean or adversarial examples.
    attack_method: If None, clean data is used. If 'PGD' or 'FGSM', adversarial attack is applied using torchattacks.
    """
    net.eval()
    correct = 0
    total = 0

    # Set up adversarial attack if specified
    if attack_method == 'PGD':
        attack = torchattacks.PGD(net, eps=0.3, alpha=2/255, steps=40)
    elif attack_method == 'FGSM':
        attack = torchattacks.FGSM(net, eps=0.3)
    else:
        attack = None

    for inputs, target in dataloader:
        inputs = inputs.view(inputs.size(0), -1).to(device)
        target = target.to(device)

        if attack:
            # Ensure inputs require gradients for adversarial attack generation
            inputs.requires_grad_()
            # Apply adversarial attack without torch.no_grad()
            inputs = attack(inputs, target) 

        # Evaluation should be inside torch.no_grad()
        with torch.no_grad():
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on {"adversarial" if attack else "clean"} examples: {accuracy:.2f}%')

# PEPITA training loop with memory optimization and hyperparameter control
def train_pepita(net, trainloader, valloader, config):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    net.to(device)
    
    # Hyperparameters from config
    epochs = config['epochs']
    eta = config['learning_rate']
    gamma = config['momentum']
    batch_size = config['batch_size']
    keep_rate = config['keep_rate']
    optimizer_type = config['optimizer_type']
    
    # Initialize feedback matrix B for MNIST
    B = initialize_feedback_matrix(28*28, 10, device=device)

    criterion = nn.CrossEntropyLoss()

    # Momentum optimizer setup
    v_w_all = [torch.zeros(w.shape).to(device) for w in net.parameters()]
    
    for epoch in range(epochs):
        if epoch in config['lr_decay_epochs']:  # Learning rate decay
            eta *= config['lr_decay_factor']
            
        running_loss = 0
        net.train()  # Set network to training mode
        for i, data in enumerate(trainloader, 0):
            inputs, target = data
            inputs = inputs.view(inputs.size(0), -1).to(device)  # Flatten MNIST input and move to device
            target = target.to(device)  # Move target to device

            target_onehot = F.one_hot(target, num_classes=10).float().to(device)

            # Dropout masks for the two forward passes
            do_masks = []
            if keep_rate < 1:
                for layer in [net.fc1, net.fc2]:
                    mask = Variable(torch.ones(inputs.shape[0], layer.out_features).bernoulli_(keep_rate)).to(device) / keep_rate
                    do_masks.append(mask)

            # Disable gradient tracking to save memory
            with torch.no_grad():

                # Forward pass 1 (standard forward pass)
                outputs = net(inputs, do_masks=do_masks)

                # Compute the error (difference between output and target)
                error_signal = outputs - target_onehot

                # Modify the input with the error and feedback matrix
                error_input = error_signal @ B.T
                mod_inputs = inputs + error_input

                # Forward pass 2 (modulated forward pass)
                mod_outputs = net(mod_inputs, do_masks=do_masks)

                # Compute weight updates
                delta_w_all = []
                for l_idx, layer in enumerate([net.fc1, net.fc2, net.fc3]):
                    if l_idx == 0:
                        delta_w = -(F.relu(net.fc1(inputs)) - F.relu(net.fc1(mod_inputs))).T @ inputs
                    elif l_idx == 1:
                        delta_w = -(F.relu(net.fc2(F.relu(net.fc1(inputs)))) - F.relu(net.fc2(F.relu(net.fc1(mod_inputs))))).T @ F.relu(net.fc1(mod_inputs))
                    else:
                        delta_w = -(error_signal.T @ F.relu(net.fc2(F.relu(net.fc1(mod_inputs)))))

                    delta_w_all.append(delta_w / batch_size)

                # Apply the weight changes
                if optimizer_type == 'SGD':
                    for l_idx, w in enumerate(net.parameters()):
                        w.data += eta * delta_w_all[l_idx]
                elif optimizer_type == 'mom':
                    for l_idx, w in enumerate(net.parameters()):
                        v_w_all[l_idx] = gamma * v_w_all[l_idx] + eta * delta_w_all[l_idx]
                        w.data += v_w_all[l_idx]

            # Clear memory to avoid memory leak
            torch.cuda.empty_cache()

            # Loss calculation
            loss = criterion(outputs, target)
            running_loss += loss.item()
            if i % 100 == 99:  # Print every 100 mini-batches
                #print(f'Epoch [{epoch+1}], Batch [{i+1}], Loss: {running_loss / 100}')
                running_loss = 0

        # Validation step at the end of each epoch
        val_loss, val_accuracy = validate_pepita(net, valloader, device, criterion)
        print(f'Validation - Epoch [{epoch+1}], Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')

# Main function to initialize and run training with validation and evaluation
def main():
    config = {
        'input_size': 28*28,  # MNIST images (28x28)
        'hidden_sizes': [256, 128],  # Hidden layer sizes
        'output_size': 10,  # Number of classes (MNIST has 10 classes)
        'epochs': 50,
        'learning_rate': 0.01,
        'momentum': 0.9,
        'batch_size': 16,
        'keep_rate': 0.9,
        'optimizer_type': 'mom',  # Options: 'SGD', 'mom'
        'lr_decay_epochs': [3],  # Learning rate decay at epoch 3
        'lr_decay_factor': 0.1  # Factor by which to decay the learning rate
    }
    
    # Initialize network with parameters from config
    net = PEPITANetwork(config['input_size'], config['hidden_sizes'], config['output_size'])
    
    # Load data
    train_loader, test_loader = get_dataloaders(config['batch_size'])
    
    # Train the model with validation
    train_pepita(net, train_loader, test_loader, config)

    # Evaluate the model on clean data
    print("Evaluating model on clean data...")
    evaluate_model(net, test_loader, torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

    # Evaluate the model on adversarial examples using PGD
    print("Evaluating model on adversarial data (PGD)...")
    evaluate_model(net, test_loader, torch.device('cuda' if torch.cuda.is_available() else 'cpu'), attack_method='PGD')

    # Evaluate the model on adversarial examples using FGSM
    print("Evaluating model on adversarial data (FGSM)...")
    evaluate_model(net, test_loader, torch.device('cuda' if torch.cuda.is_available() else 'cpu'), attack_method='FGSM')

if __name__ == "__main__":
    main()


#### This code defines a function, find_best_learning_rate, that experiments with different learning rates to find the best one based on validation accuracy.

In [ ]:
def find_best_learning_rate(learning_rates, train_loader, test_loader, config_template):
    """
    Experiment with different learning rates to find the best one based on validation accuracy.
    After finding the best model, evaluate it on clean and adversarial examples.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    best_accuracy = 0
    best_lr = None
    best_model = None
    
    for lr in learning_rates:
        print(f"Training with learning rate: {lr}")
        # Update config with the current learning rate
        config = config_template.copy()
        config['learning_rate'] = lr
        
        # Initialize network with parameters from config
        net = PEPITANetwork(config['input_size'], config['hidden_sizes'], config['output_size'])
        
        # Train the model
        train_pepita(net, train_loader, test_loader, config)
        
        # Validate the model on the clean validation set
        val_loss, val_accuracy = validate_pepita(net, test_loader, device, nn.CrossEntropyLoss())
        print(f"Validation accuracy with learning rate {lr}: {val_accuracy:.2f}%")
        
        # Check if this is the best model
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_lr = lr
            best_model = net
    
    print(f"\nBest learning rate found: {best_lr} with validation accuracy: {best_accuracy:.2f}%")
    
    # Evaluate the best model on clean data and adversarial data
    print("\nEvaluating the best model on clean data...")
    evaluate_model(best_model, test_loader, device)

    print("\nEvaluating the best model on adversarial data (PGD)...")
    evaluate_model(best_model, test_loader, device, attack_method='PGD')

    print("\nEvaluating the best model on adversarial data (FGSM)...")
    evaluate_model(best_model, test_loader, device, attack_method='FGSM')


# Define the learning rates to experiment with
learning_rates = [0.255, 0.016, 0.012, 0.029]


# Create the base configuration template
config_template = {
    'input_size': 28*28,  # MNIST images (28x28)
    'hidden_sizes': [256, 128],  # Hidden layer sizes
    'output_size': 10,  # Number of classes (MNIST has 10 classes)
    'epochs': 50,
    'learning_rate': 0.01,  # Placeholder, will be updated
    'momentum': 0.9,
    'batch_size': 16,
    'keep_rate': 0.9,
    'optimizer_type': 'mom',  # Options: 'SGD', 'mom'
    'lr_decay_epochs': [3],  # Learning rate decay at epoch 3
    'lr_decay_factor': 0.1  # Factor by which to decay the learning rate
}

# Load the data
train_loader, test_loader = get_dataloaders(config_template['batch_size'])

# Run the learning rate experiment
find_best_learning_rate(learning_rates, train_loader, test_loader, config_template)


#### This code is an extension of the previous learning rate experimentation but focuses on finding the best learning rate based on adversarial validation accuracy.

In [ ]:
def find_best_learning_rate_with_adversarial_validation(learning_rates, train_loader, test_loader, config_template, attack_method='PGD'):
    """
    Experiment with different learning rates to find the best one based on adversarial validation accuracy.
    After finding the best model, evaluate it on clean and adversarial examples.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    best_accuracy = 0
    best_lr = None
    best_model = None
    
    for lr in learning_rates:
        print(f"Training with learning rate: {lr}")
        # Update config with the current learning rate
        config = config_template.copy()
        config['learning_rate'] = lr
        
        # Initialize network with parameters from config
        net = PEPITANetwork(config['input_size'], config['hidden_sizes'], config['output_size'])
        
        # Train the model
        train_pepita(net, train_loader, test_loader, config)
        
        # Validate the model using adversarial examples
        print(f"Validating with adversarial examples ({attack_method})...")
        val_accuracy = adversarial_validation(net, test_loader, device, attack_method=attack_method)
        print(f"Adversarial validation accuracy with learning rate {lr}: {val_accuracy:.2f}%")
        
        # Check if this is the best model
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_lr = lr
            best_model = net
    
    print(f"\nBest learning rate found: {best_lr} with adversarial validation accuracy: {best_accuracy:.2f}%")
    
    # Evaluate the best model on clean data and adversarial data
    print("\nEvaluating the best model on clean data...")
    evaluate_model(best_model, test_loader, device)

    print("\nEvaluating the best model on adversarial data (PGD)...")
    evaluate_model(best_model, test_loader, device, attack_method='PGD')

    print("\nEvaluating the best model on adversarial data (FGSM)...")
    evaluate_model(best_model, test_loader, device, attack_method='FGSM')

# Function to perform adversarial validation
def adversarial_validation(net, dataloader, device, attack_method='PGD'):
    """
    Perform adversarial validation on the given dataset using the specified attack method.
    Returns the accuracy on adversarial examples.
    """
    print(f"Evaluating adversarial accuracy using {attack_method} attack...")
    correct = 0
    total = 0

    # Set up adversarial attack if specified
    if attack_method == 'PGD':
        attack = torchattacks.PGD(net, eps=0.3, alpha=2/255, steps=40)
    elif attack_method == 'FGSM':
        attack = torchattacks.FGSM(net, eps=0.3)
    else:
        raise ValueError(f"Unsupported attack method: {attack_method}")

    net.eval()  # Set network to evaluation mode

    for inputs, target in dataloader:
        inputs = inputs.view(inputs.size(0), -1).to(device)
        target = target.to(device)

        # Ensure inputs require gradients for adversarial attack generation
        inputs.requires_grad_()

        # Apply the adversarial attack
        inputs = attack(inputs, target)

        # Disable gradient computation for inference
        with torch.no_grad():
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Define the learning rates to experiment with
learning_rates = [0.378, 0.037, 0.025, 0.061]

# Create the base configuration template
config_template = {
    'input_size': 28*28,  # MNIST images (28x28)
    'hidden_sizes': [256, 128],  # Hidden layer sizes
    'output_size': 10,  # Number of classes (MNIST has 10 classes)
    'epochs': 50,
    'learning_rate': 0.01,  # Placeholder, will be updated
    'momentum': 0.9,
    'batch_size': 16,
    'keep_rate': 0.9,
    'optimizer_type': 'mom',  # Options: 'SGD', 'mom'
    'lr_decay_epochs': [3],  # Learning rate decay at epoch 3
    'lr_decay_factor': 0.1  # Factor by which to decay the learning rate
}

# Load the data
train_loader, test_loader = get_dataloaders(config_template['batch_size'])

# Run the learning rate experiment with adversarial validation
find_best_learning_rate_with_adversarial_validation(learning_rates, train_loader, test_loader, config_template, attack_method='PGD')


#### This code introduces adversarial training to enhance the model's robustness by generating adversarial samples during training and evaluates different learning rates to find the best one.

In [ ]:
# Function for adversarial training
def adversarial_train_pepita(net, trainloader, valloader, config, attack_method='PGD'):
    """
    Train the PEPITA network with adversarial samples.
    Each batch is augmented with adversarial examples generated by the specified attack method.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    net.to(device)

    # Hyperparameters from config
    epochs = config['epochs']
    eta = config['learning_rate']
    gamma = config['momentum']
    batch_size = config['batch_size']
    keep_rate = config['keep_rate']
    optimizer_type = config['optimizer_type']
    
    # Initialize feedback matrix B for MNIST
    B = initialize_feedback_matrix(28*28, 10, device=device)

    criterion = nn.CrossEntropyLoss()

    # Momentum optimizer setup
    v_w_all = [torch.zeros(w.shape).to(device) for w in net.parameters()]
    
    # Setup adversarial attack
    if attack_method == 'PGD':
        attack = torchattacks.PGD(net, eps=0.3, alpha=2/255, steps=40)
    elif attack_method == 'FGSM':
        attack = torchattacks.FGSM(net, eps=0.3)
    else:
        raise ValueError(f"Unsupported attack method: {attack_method}")
    
    for epoch in range(epochs):
        if epoch in config['lr_decay_epochs']:  # Learning rate decay
            eta *= config['lr_decay_factor']
        
        running_loss = 0
        net.train()  # Set network to training mode
        
        for i, data in enumerate(trainloader, 0):
            inputs, target = data
            inputs = inputs.view(inputs.size(0), -1).to(device)  # Flatten MNIST input
            target = target.to(device)  # Move target to device
            
            target_onehot = F.one_hot(target, num_classes=10).float().to(device)
            
            # Generate adversarial examples for the inputs
            inputs.requires_grad_()
            adv_inputs = attack(inputs, target)  # Adversarial inputs
            
            # Dropout masks for the two forward passes
            do_masks = []
            if keep_rate < 1:
                for layer in [net.fc1, net.fc2]:
                    mask = Variable(torch.ones(inputs.shape[0], layer.out_features).bernoulli_(keep_rate)).to(device) / keep_rate
                    do_masks.append(mask)
            
            # Perform forward pass on adversarial examples
            with torch.no_grad():
                outputs = net(adv_inputs, do_masks=do_masks)
                error_signal = outputs - target_onehot
                error_input = error_signal @ B.T
                mod_inputs = inputs + error_input
                mod_outputs = net(mod_inputs, do_masks=do_masks)
            
            # Compute weight updates
            delta_w_all = []
            for l_idx, layer in enumerate([net.fc1, net.fc2, net.fc3]):
                if l_idx == 0:
                    delta_w = -(F.relu(net.fc1(inputs)) - F.relu(net.fc1(mod_inputs))).T @ inputs
                elif l_idx == 1:
                    delta_w = -(F.relu(net.fc2(F.relu(net.fc1(inputs)))) - F.relu(net.fc2(F.relu(net.fc1(mod_inputs))))).T @ F.relu(net.fc1(mod_inputs))
                else:
                    delta_w = -(error_signal.T @ F.relu(net.fc2(F.relu(net.fc1(mod_inputs)))))
                delta_w_all.append(delta_w / batch_size)

            # Apply the weight changes
            if optimizer_type == 'SGD':
                for l_idx, w in enumerate(net.parameters()):
                    w.data += eta * delta_w_all[l_idx]
            elif optimizer_type == 'mom':
                for l_idx, w in enumerate(net.parameters()):
                    v_w_all[l_idx] = gamma * v_w_all[l_idx] + eta * delta_w_all[l_idx]
                    w.data += v_w_all[l_idx]

            # Clear memory to avoid memory leak
            torch.cuda.empty_cache()
            
            # Loss calculation
            loss = criterion(outputs, target)
            running_loss += loss.item()
            if i % 100 == 99:  # Print every 100 mini-batches
                #print(f'Epoch [{epoch+1}], Batch [{i+1}], Loss: {running_loss / 100}')
                running_loss = 0

        # Validation step at the end of each epoch
        val_loss, val_accuracy = validate_pepita(net, valloader, device, criterion)
        print(f'Validation - Epoch [{epoch+1}], Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')

# Function to find the best learning rate using adversarial training
def find_best_learning_rate_with_adversarial_training(learning_rates, train_loader, test_loader, config_template, attack_method='PGD'):
    """
    Experiment with different learning rates to find the best one based on natural validation accuracy.
    After finding the best model, evaluate it on clean and adversarial examples.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    best_accuracy = 0
    best_lr = None
    best_model = None
    
    for lr in learning_rates:
        print(f"\nTraining with learning rate: {lr}")
        # Update config with the current learning rate
        config = config_template.copy()
        config['learning_rate'] = lr
        
        # Initialize network with parameters from config
        net = PEPITANetwork(config['input_size'], config['hidden_sizes'], config['output_size'])
        
        # Perform adversarial training
        adversarial_train_pepita(net, train_loader, test_loader, config, attack_method=attack_method)
        
        # Validate the model on the clean validation set
        val_loss, val_accuracy = validate_pepita(net, test_loader, device, nn.CrossEntropyLoss())
        print(f"Natural validation accuracy with learning rate {lr}: {val_accuracy:.2f}%")
        
        # Check if this is the best model
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_lr = lr
            best_model = net
    
    print(f"\nBest learning rate found: {best_lr} with validation accuracy: {best_accuracy:.2f}%")
    
    # Evaluate the best model on clean data and adversarial data
    print("\nEvaluating the best model on clean data...")
    evaluate_model(best_model, test_loader, device)

    print("\nEvaluating the best model on adversarial data (PGD)...")
    evaluate_model(best_model, test_loader, device, attack_method='PGD')

    print("\nEvaluating the best model on adversarial data (FGSM)...")
    evaluate_model(best_model, test_loader, device, attack_method='FGSM')

# Define the learning rates to experiment with
learning_rates = [0.378, 0.037, 0.025, 0.061]

# Create the base configuration template
config_template = {
    'input_size': 28*28,  # MNIST images (28x28)
    'hidden_sizes': [256, 128],  # Hidden layer sizes
    'output_size': 10,  # Number of classes (MNIST has 10 classes)
    'epochs': 50,
    'learning_rate': 0.01,  # Placeholder, will be updated
    'momentum': 0.9,
    'batch_size': 16,
    'keep_rate': 0.9,
    'optimizer_type': 'mom',  # Options: 'SGD', 'mom'
    'lr_decay_epochs': [3],  # Learning rate decay at epoch 3
    'lr_decay_factor': 0.1  # Factor by which to decay the learning rate
}

# Load the data
train_loader, test_loader = get_dataloaders(config_template['batch_size'])

# Run the learning rate experiment with adversarial training
find_best_learning_rate_with_adversarial_training(learning_rates, train_loader, test_loader, config_template, attack_method='PGD')


#### This code performs fast adversarial training using the FGSM (Fast Gradient Sign Method) to improve model robustness, while experimenting with different learning rates to find the best one based on validation accuracy.

In [ ]:
# Function for fast adversarial training (FGSM)
def fast_adversarial_train_pepita(net, trainloader, valloader, config, attack_method='FGSM'):
    """
    Train the PEPITA network with FGSM adversarial samples.
    Each batch is augmented with FGSM-generated adversarial examples.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    net.to(device)

    # Hyperparameters from config
    epochs = config['epochs']
    eta = config['learning_rate']
    gamma = config['momentum']
    batch_size = config['batch_size']
    keep_rate = config['keep_rate']
    optimizer_type = config['optimizer_type']
    
    # Initialize feedback matrix B for MNIST
    B = initialize_feedback_matrix(28*28, 10, device=device)

    criterion = nn.CrossEntropyLoss()

    # Momentum optimizer setup
    v_w_all = [torch.zeros(w.shape).to(device) for w in net.parameters()]
    
    # Setup adversarial attack (FGSM for fast adversarial training)
    attack = torchattacks.FGSM(net, eps=0.3)
    
    for epoch in range(epochs):
        if epoch in config['lr_decay_epochs']:  # Learning rate decay
            eta *= config['lr_decay_factor']
        
        running_loss = 0
        net.train()  # Set network to training mode
        
        for i, data in enumerate(trainloader, 0):
            inputs, target = data
            inputs = inputs.view(inputs.size(0), -1).to(device)  # Flatten MNIST input
            target = target.to(device)  # Move target to device
            
            target_onehot = F.one_hot(target, num_classes=10).float().to(device)
            
            # Generate FGSM adversarial examples for the inputs
            inputs.requires_grad_()
            adv_inputs = attack(inputs, target)  # FGSM adversarial inputs
            
            # Dropout masks for the two forward passes
            do_masks = []
            if keep_rate < 1:
                for layer in [net.fc1, net.fc2]:
                    mask = Variable(torch.ones(inputs.shape[0], layer.out_features).bernoulli_(keep_rate)).to(device) / keep_rate
                    do_masks.append(mask)
            
            # Perform forward pass on adversarial examples
            with torch.no_grad():
                outputs = net(adv_inputs, do_masks=do_masks)
                error_signal = outputs - target_onehot
                error_input = error_signal @ B.T
                mod_inputs = inputs + error_input
                mod_outputs = net(mod_inputs, do_masks=do_masks)
            
            # Compute weight updates
            delta_w_all = []
            for l_idx, layer in enumerate([net.fc1, net.fc2, net.fc3]):
                if l_idx == 0:
                    delta_w = -(F.relu(net.fc1(inputs)) - F.relu(net.fc1(mod_inputs))).T @ inputs
                elif l_idx == 1:
                    delta_w = -(F.relu(net.fc2(F.relu(net.fc1(inputs)))) - F.relu(net.fc2(F.relu(net.fc1(mod_inputs))))).T @ F.relu(net.fc1(mod_inputs))
                else:
                    delta_w = -(error_signal.T @ F.relu(net.fc2(F.relu(net.fc1(mod_inputs)))))
                delta_w_all.append(delta_w / batch_size)

            # Apply the weight changes
            if optimizer_type == 'SGD':
                for l_idx, w in enumerate(net.parameters()):
                    w.data += eta * delta_w_all[l_idx]
            elif optimizer_type == 'mom':
                for l_idx, w in enumerate(net.parameters()):
                    v_w_all[l_idx] = gamma * v_w_all[l_idx] + eta * delta_w_all[l_idx]
                    w.data += v_w_all[l_idx]

            # Clear memory to avoid memory leak
            torch.cuda.empty_cache()
            
            # Loss calculation
            loss = criterion(outputs, target)
            running_loss += loss.item()
            if i % 100 == 99:  # Print every 100 mini-batches
                #print(f'Epoch [{epoch+1}], Batch [{i+1}], Loss: {running_loss / 100}')
                running_loss = 0

        # Validation step at the end of each epoch
        val_loss, val_accuracy = validate_pepita(net, valloader, device, criterion)
        print(f'Validation - Epoch [{epoch+1}], Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')


# Function to find the best learning rate with fast adversarial training
def find_best_learning_rate_fast_adversarial(learning_rates, train_loader, test_loader, config_template, attack_method='FGSM'):
    """
    Experiment with different learning rates to find the best one based on natural validation accuracy.
    After finding the best model, evaluate it on clean and adversarial examples.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    best_accuracy = 0
    best_lr = None
    best_model = None
    
    for lr in learning_rates:
        print(f"\nTraining with learning rate: {lr}")
        # Update config with the current learning rate
        config = config_template.copy()
        config['learning_rate'] = lr
        
        # Initialize network with parameters from config
        net = PEPITANetwork(config['input_size'], config['hidden_sizes'], config['output_size'])
        
        # Perform fast adversarial training (with FGSM)
        fast_adversarial_train_pepita(net, train_loader, test_loader, config, attack_method=attack_method)
        
        # Validate the model on the clean validation set
        val_loss, val_accuracy = validate_pepita(net, test_loader, device, nn.CrossEntropyLoss())
        print(f"Natural validation accuracy with learning rate {lr}: {val_accuracy:.2f}%")
        
        # Check if this is the best model
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_lr = lr
            best_model = net
    
    print(f"\nBest learning rate found: {best_lr} with natural validation accuracy: {best_accuracy:.2f}%")
    
    # Evaluate the best model on clean data and adversarial data
    print("\nEvaluating the best model on clean data...")
    evaluate_model(best_model, test_loader, device)

    print("\nEvaluating the best model on adversarial data (PGD)...")
    evaluate_model(best_model, test_loader, device, attack_method='PGD')

    print("\nEvaluating the best model on adversarial data (FGSM)...")
    evaluate_model(best_model, test_loader, device, attack_method='FGSM')

# Define the learning rates to experiment with
learning_rates = [0.097, 0.027, 0.016, 0.041]

# Create the base configuration template
config_template = {
    'input_size': 28*28,  # MNIST images (28x28)
    'hidden_sizes': [256, 128],  # Hidden layer sizes
    'output_size': 10,  # Number of classes (MNIST has 10 classes)
    'epochs': 50,
    'learning_rate': 0.01,  # Placeholder, will be updated
    'momentum': 0.9,
    'batch_size': 16,
    'keep_rate': 0.9,
    'optimizer_type': 'mom',  # Options: 'SGD', 'mom'
    'lr_decay_epochs': [3],  # Learning rate decay at epoch 3
    'lr_decay_factor': 0.1  # Factor by which to decay the learning rate
}

# Load the data
train_loader, test_loader = get_dataloaders(config_template['batch_size'])

# Run the learning rate experiment with fast adversarial training (FGSM)
find_best_learning_rate_fast_adversarial(learning_rates, train_loader, test_loader, config_template, attack_method='FGSM')


# BP

#### This code implements a backpropagation neural network using PyTorch to classify MNIST dataset images, and includes evaluation on both clean and adversarial examples.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import numpy as np
import torch.cuda
import torchattacks

# Define the backpropagation network
class BackpropNetwork(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate=0.1):
        super(BackpropNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0], bias=False)
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1], bias=False)
        self.fc3 = nn.Linear(hidden_sizes[1], output_size, bias=False)
        self.dropout_rate = dropout_rate
        
        # He initialization
        fc1_limit = np.sqrt(6.0 / input_size)
        torch.nn.init.uniform_(self.fc1.weight, a=-fc1_limit, b=fc1_limit)
        fc2_limit = np.sqrt(6.0 / hidden_sizes[0])
        torch.nn.init.uniform_(self.fc2.weight, a=-fc2_limit, b=fc2_limit)
        fc3_limit = np.sqrt(6.0 / hidden_sizes[1])
        torch.nn.init.uniform_(self.fc3.weight, a=-fc3_limit, b=fc3_limit)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x

# Setup data loaders (MNIST dataset)
def get_dataloaders(batch_size=16):
    transform = transforms.Compose([transforms.ToTensor()])
    trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
    testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)
    return trainloader, testloader

# Validation function to evaluate model performance
def validate_model(net, dataloader, device, criterion):
    net.eval()  # Set network to evaluation mode
    running_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, target in dataloader:
            inputs = inputs.view(inputs.size(0), -1).to(device)  # Flatten inputs
            target = target.to(device)
            outputs = net(inputs)  # Forward pass
            loss = criterion(outputs, target)
            running_loss += loss.item()

            # Accuracy calculation
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    
    accuracy = 100 * correct / total
    return running_loss / len(dataloader), accuracy

# Adversarial attack evaluation function
def evaluate_model(net, dataloader, device, attack_method=None):
    net.eval()
    correct = 0
    total = 0

    # Set up adversarial attack if specified
    if attack_method == 'PGD':
        attack = torchattacks.PGD(net, eps=0.3, alpha=2/255, steps=40)
    elif attack_method == 'FGSM':
        attack = torchattacks.FGSM(net, eps=0.3)
    else:
        attack = None

    for inputs, target in dataloader:
        inputs = inputs.view(inputs.size(0), -1).to(device)
        target = target.to(device)

        if attack:
            inputs.requires_grad_()
            inputs = attack(inputs, target) 

        with torch.no_grad():
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on {"adversarial" if attack else "clean"} examples: {accuracy:.2f}%')

# Backpropagation training loop
def train_backprop(net, trainloader, valloader, config):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    net.to(device)

    # Hyperparameters from config
    epochs = config['epochs']
    eta = config['learning_rate']
    batch_size = config['batch_size']

    # Use a standard optimizer like SGD with momentum
    if config['optimizer_type'] == 'SGD':
        optimizer = torch.optim.SGD(net.parameters(), lr=eta, momentum=config['momentum'])
    elif config['optimizer_type'] == 'Adam':
        optimizer = torch.optim.Adam(net.parameters(), lr=eta)

    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        if epoch in config['lr_decay_epochs']:  # Learning rate decay
            for param_group in optimizer.param_groups:
                param_group['lr'] *= config['lr_decay_factor']

        running_loss = 0
        net.train()  # Set network to training mode
        for i, data in enumerate(trainloader, 0):
            inputs, target = data
            inputs = inputs.view(inputs.size(0), -1).to(device)  # Flatten MNIST input and move to device
            target = target.to(device)  # Move target to device

            # Forward pass
            outputs = net(inputs)

            # Compute loss
            loss = criterion(outputs, target)

            # Zero gradients, perform backward pass, and update weights
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:  # Print every 100 mini-batches
                running_loss = 0

        # Validation step at the end of each epoch
        val_loss, val_accuracy = validate_model(net, valloader, device, criterion)
        print(f'Validation - Epoch [{epoch+1}], Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')

# Main function to initialize and run training with validation and evaluation
def main():
    config = {
        'input_size': 28*28,  # MNIST images (28x28)
        'hidden_sizes': [256, 128],  # Hidden layer sizes
        'output_size': 10,  # Number of classes (MNIST has 10 classes)
        'epochs': 50,
        'learning_rate': 0.01,
        'momentum': 0.9,
        'batch_size': 16,
        'optimizer_type': 'SGD',  # Options: 'SGD', 'Adam'
        'lr_decay_epochs': [3],  # Learning rate decay at epoch 3
        'lr_decay_factor': 0.1  # Factor by which to decay the learning rate
    }
    
    # Initialize network with parameters from config
    net = BackpropNetwork(config['input_size'], config['hidden_sizes'], config['output_size'])
    
    # Load data
    train_loader, test_loader = get_dataloaders(config['batch_size'])
    
    # Train the model with validation
    train_backprop(net, train_loader, test_loader, config)

    # Evaluate the model on clean data
    print("Evaluating model on clean data...")
    evaluate_model(net, test_loader, torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

    # Evaluate the model on adversarial examples using PGD
    print("Evaluating model on adversarial data (PGD)...")
    evaluate_model(net, test_loader, torch.device('cuda' if torch.cuda.is_available() else 'cpu'), attack_method='PGD')

    # Evaluate the model on adversarial examples using FGSM
    print("Evaluating model on adversarial data (FGSM)...")
    evaluate_model(net, test_loader, torch.device('cuda' if torch.cuda.is_available() else 'cpu'), attack_method='FGSM')

if __name__ == "__main__":
    main()


#### This code performs a learning rate search for the Backpropagation Network to determine the best learning rate based on validation accuracy.

In [ ]:
def find_best_learning_rate(learning_rates, train_loader, test_loader, config_template):
    """
    Experiment with different learning rates to find the best one based on validation accuracy.
    After finding the best model, evaluate it on clean and adversarial examples.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    best_accuracy = 0
    best_lr = None
    best_model = None
    
    for lr in learning_rates:
        print(f"Training with learning rate: {lr}")
        # Update config with the current learning rate
        config = config_template.copy()
        config['learning_rate'] = lr
        
        # Initialize network with parameters from config
        net = BackpropNetwork(config['input_size'], config['hidden_sizes'], config['output_size'])
        
        # Train the model
        train_backprop(net, train_loader, test_loader, config)
        
        # Validate the model on the clean validation set
        val_loss, val_accuracy = validate_model(net, test_loader, device, nn.CrossEntropyLoss())
        print(f"Validation accuracy with learning rate {lr}: {val_accuracy:.2f}%")
        
        # Check if this is the best model
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_lr = lr
            best_model = net
    
    print(f"\nBest learning rate found: {best_lr} with validation accuracy: {best_accuracy:.2f}%")
    
    # Evaluate the best model on clean data and adversarial data
    print("\nEvaluating the best model on clean data...")
    evaluate_model(best_model, test_loader, device)

    print("\nEvaluating the best model on adversarial data (PGD)...")
    evaluate_model(best_model, test_loader, device, attack_method='PGD')

    print("\nEvaluating the best model on adversarial data (FGSM)...")
    evaluate_model(best_model, test_loader, device, attack_method='FGSM')


# Define the learning rates to experiment with
learning_rates = [0.123, 0.051, 0.008, 0.035]


# Create the base configuration template
config_template = {
    'input_size': 28*28,  # MNIST images (28x28)
    'hidden_sizes': [256, 128],  # Hidden layer sizes
    'output_size': 10,  # Number of classes (MNIST has 10 classes)
    'epochs': 50,
    'learning_rate': 0.01,  # Placeholder, will be updated
    'momentum': 0.9,
    'batch_size': 16,
    'optimizer_type': 'SGD',  # Options: 'SGD', 'Adam'
    'lr_decay_epochs': [3],  # Learning rate decay at epoch 3
    'lr_decay_factor': 0.1  # Factor by which to decay the learning rate
}

# Load the data
train_loader, test_loader = get_dataloaders(config_template['batch_size'])

# Run the learning rate experiment
find_best_learning_rate(learning_rates, train_loader, test_loader, config_template)


#### This code extends the learning rate search by incorporating adversarial validation to find the best learning rate for a model that can resist adversarial attacks. The key additions are the validation on adversarial examples and the evaluation of model performance based on adversarial accuracy.

In [ ]:
def find_best_learning_rate_with_adversarial_validation(learning_rates, train_loader, test_loader, config_template, attack_method='PGD'):
    """
    Experiment with different learning rates to find the best one based on adversarial validation accuracy.
    After finding the best model, evaluate it on clean and adversarial examples.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    best_accuracy = 0
    best_lr = None
    best_model = None
    
    for lr in learning_rates:
        print(f"Training with learning rate: {lr}")
        # Update config with the current learning rate
        config = config_template.copy()
        config['learning_rate'] = lr
        
        # Initialize network with parameters from config
        net = BackpropNetwork(config['input_size'], config['hidden_sizes'], config['output_size'])
        
        # Train the model
        train_backprop(net, train_loader, test_loader, config)
        
        # Validate the model using adversarial examples
        print(f"Validating with adversarial examples ({attack_method})...")
        val_accuracy = adversarial_validation(net, test_loader, device, attack_method=attack_method)
        print(f"Adversarial validation accuracy with learning rate {lr}: {val_accuracy:.2f}%")
        
        # Check if this is the best model
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_lr = lr
            best_model = net
    
    print(f"\nBest learning rate found: {best_lr} with adversarial validation accuracy: {best_accuracy:.2f}%")
    
    # Evaluate the best model on clean data and adversarial data
    print("\nEvaluating the best model on clean data...")
    evaluate_model(best_model, test_loader, device)

    print("\nEvaluating the best model on adversarial data (PGD)...")
    evaluate_model(best_model, test_loader, device, attack_method='PGD')

    print("\nEvaluating the best model on adversarial data (FGSM)...")
    evaluate_model(best_model, test_loader, device, attack_method='FGSM')

# Function to perform adversarial validation
def adversarial_validation(net, dataloader, device, attack_method='PGD'):
    """
    Perform adversarial validation on the given dataset using the specified attack method.
    Returns the accuracy on adversarial examples.
    """
    print(f"Evaluating adversarial accuracy using {attack_method} attack...")
    correct = 0
    total = 0

    # Set up adversarial attack if specified
    if attack_method == 'PGD':
        attack = torchattacks.PGD(net, eps=0.3, alpha=2/255, steps=40)
    elif attack_method == 'FGSM':
        attack = torchattacks.FGSM(net, eps=0.3)
    else:
        raise ValueError(f"Unsupported attack method: {attack_method}")

    net.eval()  # Set network to evaluation mode

    for inputs, target in dataloader:
        inputs = inputs.view(inputs.size(0), -1).to(device)
        target = target.to(device)

        # Ensure inputs require gradients for adversarial attack generation
        inputs.requires_grad_()

        # Apply the adversarial attack
        inputs = attack(inputs, target)

        # Disable gradient computation for inference
        with torch.no_grad():
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Define the learning rates to experiment with
learning_rates = [0.378, 0.273, 0.039, 0.180]

# Create the base configuration template
config_template = {
    'input_size': 28*28,  # MNIST images (28x28)
    'hidden_sizes': [256, 128],  # Hidden layer sizes
    'output_size': 10,  # Number of classes (MNIST has 10 classes)
    'epochs': 50,
    'learning_rate': 0.01,  # Placeholder, will be updated
    'momentum': 0.9,
    'batch_size': 16,
    'optimizer_type': 'SGD',  # Options: 'SGD', 'Adam'
    'lr_decay_epochs': [3],  # Learning rate decay at epoch 3
    'lr_decay_factor': 0.1  # Factor by which to decay the learning rate
}

# Load the data
train_loader, test_loader = get_dataloaders(config_template['batch_size'])

# Run the learning rate experiment with adversarial validation
find_best_learning_rate_with_adversarial_validation(learning_rates, train_loader, test_loader, config_template, attack_method='PGD')


#### This code introduces adversarial training using either the PGD (Projected Gradient Descent) or FGSM (Fast Gradient Sign Method) attack methods to improve the model's robustness. The function find_best_learning_rate_with_adversarial_training is used to identify the best learning rate based on validation accuracy

In [ ]:
import torchattacks

# Function for adversarial training using backpropagation
def adversarial_train_backprop(net, trainloader, valloader, config, attack_method='PGD'):
    """
    Train the network with adversarial samples using backpropagation.
    Each batch is augmented with adversarial examples generated by the specified attack method.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    net.to(device)

    # Hyperparameters from config
    epochs = config['epochs']
    eta = config['learning_rate']
    batch_size = config['batch_size']

    # Use a standard optimizer like SGD with momentum
    if config['optimizer_type'] == 'SGD':
        optimizer = torch.optim.SGD(net.parameters(), lr=eta, momentum=config['momentum'])
    elif config['optimizer_type'] == 'Adam':
        optimizer = torch.optim.Adam(net.parameters(), lr=eta)

    criterion = nn.CrossEntropyLoss()

    # Setup adversarial attack
    if attack_method == 'PGD':
        attack = torchattacks.PGD(net, eps=0.3, alpha=2/255, steps=40)
    elif attack_method == 'FGSM':
        attack = torchattacks.FGSM(net, eps=0.3)
    else:
        raise ValueError(f"Unsupported attack method: {attack_method}")

    for epoch in range(epochs):
        if epoch in config['lr_decay_epochs']:  # Learning rate decay
            for param_group in optimizer.param_groups:
                param_group['lr'] *= config['lr_decay_factor']
        
        running_loss = 0
        net.train()  # Set network to training mode
        
        for i, data in enumerate(trainloader, 0):
            inputs, target = data
            inputs = inputs.view(inputs.size(0), -1).to(device)  # Flatten MNIST input
            target = target.to(device)  # Move target to device
            
            # Generate adversarial examples for the inputs
            inputs.requires_grad_()
            adv_inputs = attack(inputs, target)  # Adversarial inputs
            
            # Perform forward pass on adversarial examples
            optimizer.zero_grad()  # Zero out the previous gradients
            outputs = net(adv_inputs)
            loss = criterion(outputs, target)  # Compute loss
            loss.backward()  # Backpropagate the loss
            optimizer.step()  # Update weights

            running_loss += loss.item()
            if i % 100 == 99:  # Print every 100 mini-batches
                #print(f'Epoch [{epoch+1}], Batch [{i+1}], Loss: {running_loss / 100:.4f}')
                running_loss = 0

        # Validation step at the end of each epoch
        val_loss, val_accuracy = validate_model(net, valloader, device, criterion)
        print(f'Validation - Epoch [{epoch+1}], Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')

# Function to find the best learning rate using adversarial training
def find_best_learning_rate_with_adversarial_training(learning_rates, train_loader, test_loader, config_template, attack_method='PGD'):
    """
    Experiment with different learning rates to find the best one based on natural validation accuracy.
    After finding the best model, evaluate it on clean and adversarial examples.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    best_accuracy = 0
    best_lr = None
    best_model = None
    
    for lr in learning_rates:
        print(f"\nTraining with learning rate: {lr}")
        # Update config with the current learning rate
        config = config_template.copy()
        config['learning_rate'] = lr
        
        # Initialize network with parameters from config
        net = BackpropNetwork(config['input_size'], config['hidden_sizes'], config['output_size'])
        
        # Perform adversarial training
        adversarial_train_backprop(net, train_loader, test_loader, config, attack_method=attack_method)
        
        # Validate the model on the clean validation set
        val_loss, val_accuracy = validate_model(net, test_loader, device, nn.CrossEntropyLoss())
        print(f"Natural validation accuracy with learning rate {lr}: {val_accuracy:.2f}%")
        
        # Check if this is the best model
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_lr = lr
            best_model = net
    
    print(f"\nBest learning rate found: {best_lr} with validation accuracy: {best_accuracy:.2f}%")
    
    # Evaluate the best model on clean data and adversarial data
    print("\nEvaluating the best model on clean data...")
    evaluate_model(best_model, test_loader, device)

    print("\nEvaluating the best model on adversarial data (PGD)...")
    evaluate_model(best_model, test_loader, device, attack_method='PGD')

    print("\nEvaluating the best model on adversarial data (FGSM)...")
    evaluate_model(best_model, test_loader, device, attack_method='FGSM')

# Define the learning rates to experiment with
learning_rates = [0.052, 0.030, 0.012, 0.014]

# Create the base configuration template
config_template = {
    'input_size': 28*28,  # MNIST images (28x28)
    'hidden_sizes': [256, 128],  # Hidden layer sizes
    'output_size': 10,  # Number of classes (MNIST has 10 classes)
    'epochs': 50,
    'learning_rate': 0.01,  # Placeholder, will be updated
    'momentum': 0.9,
    'batch_size': 16,
    'optimizer_type': 'SGD',  # Options: 'SGD', 'Adam'
    'lr_decay_epochs': [3],  # Learning rate decay at epoch 3
    'lr_decay_factor': 0.1  # Factor by which to decay the learning rate
}

# Load the data
train_loader, test_loader = get_dataloaders(config_template['batch_size'])

# Run the learning rate experiment with adversarial training
find_best_learning_rate_with_adversarial_training(learning_rates, train_loader, test_loader, config_template, attack_method='PGD')


#### This code performs fast adversarial training using FGSM (Fast Gradient Sign Method) with backpropagation, and experiments with different learning rates to find the best one. It evaluates the model based on both natural (clean) and adversarial examples.

In [ ]:
import torchattacks

# Function for fast adversarial training (FGSM) using backpropagation
def fast_adversarial_train_backprop(net, trainloader, valloader, config, attack_method='FGSM'):
    """
    Train the network with FGSM adversarial samples using backpropagation.
    Each batch is augmented with FGSM-generated adversarial examples.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    net.to(device)

    # Hyperparameters from config
    epochs = config['epochs']
    eta = config['learning_rate']
    batch_size = config['batch_size']

    # Use a standard optimizer like SGD with momentum
    if config['optimizer_type'] == 'SGD':
        optimizer = torch.optim.SGD(net.parameters(), lr=eta, momentum=config['momentum'])
    elif config['optimizer_type'] == 'Adam':
        optimizer = torch.optim.Adam(net.parameters(), lr=eta)

    criterion = nn.CrossEntropyLoss()

    # Setup adversarial attack (FGSM for fast adversarial training)
    attack = torchattacks.FGSM(net, eps=0.3)

    for epoch in range(epochs):
        if epoch in config['lr_decay_epochs']:  # Learning rate decay
            for param_group in optimizer.param_groups:
                param_group['lr'] *= config['lr_decay_factor']

        running_loss = 0
        net.train()  # Set network to training mode
        
        for i, data in enumerate(trainloader, 0):
            inputs, target = data
            inputs = inputs.view(inputs.size(0), -1).to(device)  # Flatten MNIST input
            target = target.to(device)  # Move target to device
            
            # Generate FGSM adversarial examples for the inputs
            inputs.requires_grad_()
            adv_inputs = attack(inputs, target)  # FGSM adversarial inputs
            
            # Perform forward pass on adversarial examples
            optimizer.zero_grad()  # Zero out previous gradients
            outputs = net(adv_inputs)  # Forward pass with adversarial inputs
            loss = criterion(outputs, target)  # Compute loss
            loss.backward()  # Backpropagate the loss
            optimizer.step()  # Update weights

            running_loss += loss.item()
            if i % 100 == 99:  # Print every 100 mini-batches
                #print(f'Epoch [{epoch+1}], Batch [{i+1}], Loss: {running_loss / 100:.4f}')
                running_loss = 0

        # Validation step at the end of each epoch
        val_loss, val_accuracy = validate_model(net, valloader, device, criterion)
        print(f'Validation - Epoch [{epoch+1}], Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.2f}%')


# Function to find the best learning rate with fast adversarial training (FGSM)
def find_best_learning_rate_fast_adversarial(learning_rates, train_loader, test_loader, config_template, attack_method='FGSM'):
    """
    Experiment with different learning rates to find the best one based on natural validation accuracy.
    After finding the best model, evaluate it on clean and adversarial examples.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    best_accuracy = 0
    best_lr = None
    best_model = None
    
    for lr in learning_rates:
        print(f"\nTraining with learning rate: {lr}")
        # Update config with the current learning rate
        config = config_template.copy()
        config['learning_rate'] = lr
        
        # Initialize network with parameters from config
        net = BackpropNetwork(config['input_size'], config['hidden_sizes'], config['output_size'])
        
        # Perform fast adversarial training (with FGSM)
        fast_adversarial_train_backprop(net, train_loader, test_loader, config, attack_method=attack_method)
        
        # Validate the model on the clean validation set
        val_loss, val_accuracy = validate_model(net, test_loader, device, nn.CrossEntropyLoss())
        print(f"Natural validation accuracy with learning rate {lr}: {val_accuracy:.2f}%")
        
        # Check if this is the best model
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_lr = lr
            best_model = net
    
    print(f"\nBest learning rate found: {best_lr} with natural validation accuracy: {best_accuracy:.2f}%")
    
    # Evaluate the best model on clean data and adversarial data
    print("\nEvaluating the best model on clean data...")
    evaluate_model(best_model, test_loader, device)

    print("\nEvaluating the best model on adversarial data (PGD)...")
    evaluate_model(best_model, test_loader, device, attack_method='PGD')

    print("\nEvaluating the best model on adversarial data (FGSM)...")
    evaluate_model(best_model, test_loader, device, attack_method='FGSM')


# Define the learning rates to experiment with
learning_rates = [0.097, 0.010, 0.012, 0.027]

# Create the base configuration template
config_template = {
    'input_size': 28*28,  # MNIST images (28x28)
    'hidden_sizes': [256, 128],  # Hidden layer sizes
    'output_size': 10,  # Number of classes (MNIST has 10 classes)
    'epochs': 50,
    'learning_rate': 0.01,  # Placeholder, will be updated
    'momentum': 0.9,
    'batch_size': 16,
    'optimizer_type': 'SGD',  # Options: 'SGD', 'Adam'
    'lr_decay_epochs': [3],  # Learning rate decay at epoch 3
    'lr_decay_factor': 0.1  # Factor by which to decay the learning rate
}

# Load the data
train_loader, test_loader = get_dataloaders(config_template['batch_size'])

# Run the learning rate experiment with fast adversarial training (FGSM)
find_best_learning_rate_fast_adversarial(learning_rates, train_loader, test_loader, config_template, attack_method='FGSM')
